In [ ]:
## !pip install obspy
## !pip install geopy

   ---------------------------------------- 0.0/125.4 kB ? eta -:--:--
   --- ------------------------------------ 10.2/125.4 kB ? eta -:--:--
   ------------ -------------------------- 41.0/125.4 kB 495.5 kB/s eta 0:00:01
   ---------------------------------- --- 112.6/125.4 kB 939.4 kB/s eta 0:00:01
   -------------------------------------  122.9/125.4 kB 722.1 kB/s eta 0:00:01
   -------------------------------------- 125.4/125.4 kB 616.0 kB/s eta 0:00:00
   ---------------------------------------- 0.0/40.3 kB ? eta -:--:--
   ---------------------------------------- 40.3/40.3 kB 970.5 kB/s eta 0:00:00


In [13]:
from obspy.clients.fdsn import Client
from geopy.geocoders import Nominatim


client = Client("USGS")

geolocator = Nominatim(user_agent="TomPEQProject")
city = "San Francisco, CA"
loc = geolocator.geocode(city)
maxradius = 1.0  # Maximum radius in degrees (approx. 111 km per degree)


try:
    catalog = client.get_events(starttime="1900-10-01", 
                                endtime="2024-10-11", 
                                minmagnitude=2.5,
                                eventtype="earthquake",
                                latitude=loc.latitude,
                                longitude=loc.longitude,
                                maxradius=maxradius
                                )
except Exception as e:
    print(f"Error fetching events: {e}")

for event in catalog:
    temp = event.origins[0]
    location = temp.longitude, temp.latitude, temp.depth
    print(f"Event: {event}, Magnitude: {event.magnitudes[0].mag}")
    print(location)


Event: Event:	2024-10-05T21:47:59.260000Z | +38.521, -122.438 | 2.52 md | manual

	            resource_id: ResourceIdentifier(id="quakeml:earthquake.usgs.gov/fdsnws/event/1/query?eventid=nc75070886&format=quakeml")
	             event_type: 'earthquake'
	          creation_info: CreationInfo(agency_id='nc', creation_time=UTCDateTime(2024, 10, 27, 2, 41, 5, 40000), version='2')
	    preferred_origin_id: ResourceIdentifier(id="quakeml:earthquake.usgs.gov/product/origin/nc75070886/nc/1728603307860/product.xml")
	 preferred_magnitude_id: ResourceIdentifier(id="quakeml:earthquake.usgs.gov/product/origin/nc75070886/nc/1728603307860/product.xml#magnitude")
	                   ---------
	     event_descriptions: 1 Elements
	                origins: 1 Elements
	             magnitudes: 1 Elements, Magnitude: 2.52
(-122.438166666667, 38.5208333333333, 9890.0)
Event: Event:	2024-09-13T22:15:47.900000Z | +37.410, -121.759 | 3.37 ml | manual

	            resource_id: ResourceIdentifier(id="quakem